# Genie 일간 Top100 크롤링

In [56]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import time, random

In [57]:
import requests

# 크롤링

## 크롤링할 링크 생성

In [58]:
# 날짜 리스트로 출력
start_date ='2022-01-01'
end_date = '2022-03-31'
dates = pd.date_range(start_date, end_date).to_list()
dates[:5]

[Timestamp('2022-01-01 00:00:00', freq='D'),
 Timestamp('2022-01-02 00:00:00', freq='D'),
 Timestamp('2022-01-03 00:00:00', freq='D'),
 Timestamp('2022-01-04 00:00:00', freq='D'),
 Timestamp('2022-01-05 00:00:00', freq='D')]

In [59]:
# 순위 날짜
rank_date_list = []
for one in dates:
    temp = str(one).split(' ')[0]
    rank_date_list.append(temp)

len(rank_date_list)

90

In [60]:
rank_date_list[:5]

['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05']

In [61]:
# 크롤링할 날짜 생성
date_list = []
for one in dates:
    temp = str(one).split(' ')[0].replace('-', '')
    date_list.append(temp)

len(date_list)

90

In [62]:
date_list[:5]

['20220101', '20220102', '20220103', '20220104', '20220105']

In [63]:
# 링크 리스트
link_list = []
for one in date_list:
    temp1 = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={one}&hh=15&rtm=N&pg=1'
    temp2 = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={one}&hh=15&rtm=N&pg=2'
    link_list.append(temp1)
    link_list.append(temp2)

len(link_list)

180

In [64]:
link_list[:5]

['https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220101&hh=15&rtm=N&pg=1',
 'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220101&hh=15&rtm=N&pg=2',
 'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220102&hh=15&rtm=N&pg=1',
 'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220102&hh=15&rtm=N&pg=2',
 'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220103&hh=15&rtm=N&pg=1']

## 확인

In [65]:
# 페이지 접속
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
# 지니뮤직 url
data = requests.get(link_list[1], headers=headers)
# soup : html형식
soup = bs(data.text, 'lxml')

In [66]:
soup.title.text

'음악, 그리고 설레임 - 지니'

In [67]:
# 곡 제목
song_title_list = []
song_info = soup.find_all('td', 'info')
for one in song_info:
    temp = one.find('a', 'title ellipsis').text.strip()
    song_title_list.append(temp)

In [73]:
type(song_title_list[3])

str

In [74]:
song_title_list[3]

'19금\n                                            \n                                        \n                                        \n                                    \n                                    Peaches (Feat. Daniel Caesar & Giveon)'

In [77]:
song_title_list[3][3:].strip()

'Peaches (Feat. Daniel Caesar & Giveon)'

In [78]:
# 곡 제목
song_title_list = []
song_info = soup.find_all('td', 'info')
for one in song_info:
    temp = one.find('a', 'title ellipsis').text.strip()
    if '19금' in temp:
        temp = temp[3:].strip()
    song_title_list.append(temp)

In [81]:
song_title_list[3]

'Peaches (Feat. Daniel Caesar & Giveon)'

## 크롤링 준비

In [82]:
len(link_list)/2

90.0

In [83]:
rank_list = []
for i in range(1, 101):
    rank_list.append(i)

## 파일로 출력

In [85]:
num = int(len(link_list)/2)

for i in tqdm(range(num)):
    first = i*2
    second = i*2 + 1

    first_link = link_list[first]
    second_link = link_list[second]

    song_title_list = []
    artist_list = []
    album_list = []

    # 페이지 접속
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    # 지니뮤직 url
    data = requests.get(first_link, headers=headers)
    # soup : html형식
    soup = bs(data.text, 'lxml')

    # 곡 제목
    song_info = soup.find_all('td', 'info')
    for one in song_info:
        temp = one.find('a', 'title ellipsis').text.strip()
        if '19금' in temp:
            temp = temp[3:].strip()
        song_title_list.append(temp)
    
    # 가수
    for one in song_info:
        temp = one.find('a', 'artist ellipsis').text
        artist_list.append(temp)

    # 앨범
    for one in song_info:
        temp = one.find('a', 'albumtitle ellipsis').text
        album_list.append(temp)

    # 페이지 접속
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    # 지니뮤직 url
    data = requests.get(second_link, headers=headers)
    # soup : html형식
    soup = bs(data.text, 'lxml')

    # 곡 제목
    song_info = soup.find_all('td', 'info')
    for one in song_info:
        temp = one.find('a', 'title ellipsis').text.strip()
        if '19금' in temp:
            temp = temp[3:].strip()
        song_title_list.append(temp)
    
    # 가수
    for one in song_info:
        temp = one.find('a', 'artist ellipsis').text
        artist_list.append(temp)

    # 앨범
    for one in song_info:
        temp = one.find('a', 'albumtitle ellipsis').text
        album_list.append(temp)

    # 날짜
    temp = rank_date_list[i]
    repeat_rank_date_list = []
    for j in range(100):
        repeat_rank_date_list.append(temp)

    # 데이터 셋으로 만들기
    dict = {'날짜':repeat_rank_date_list, '순위':rank_list, '곡':song_title_list, '가수':artist_list, '앨범':album_list}
    df = pd.DataFrame(dict)

    # excel 파일로 출력
    date = date_list[i]
    df.to_excel(f'../crawling_data/day_genie/day_genie_{date}.xlsx', index=False)

    time.sleep( random.uniform(0.5, 0.9) )

100%|██████████| 90/90 [01:45<00:00,  1.17s/it]


# 코드 마지막